In [1]:
import requests
import pandas as pd
URL="https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"

In [2]:
page=requests.get(URL)

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
poke_content=soup.find(id='mw-content-text')

poke_tables=poke_content.find_all('table')

In [5]:
def generate_pd(select_generation):
    gen_list=poke_tables[select_generation]
    poke_info=gen_list.contents[1]    
    info_start = 1
# place where to get the pokemon info
    info_row=gen_list.contents[info_start]

    for pokemon_info_values, even_index_chec in zip(info_row.contents, range(0,len(info_row.contents))):
    # Pokemons' values are stored in even index (divisible by 2 and is not 0)
      if ((even_index_chec % 2) == 0) & (even_index_chec != 0) :
        pokemon_raw_info = pokemon_info_values.text.strip().split('\n')

        
        ## Pokemons that are other regional forms and has 1 type
        if len(pokemon_raw_info) == 7:
            ldex = pokemon_raw_info[0]
            ndex = pokemon_raw_info[1]
            poke_name = pokemon_raw_info[4]
            type1 = pokemon_raw_info[6]
            type2 = ''
            categ = 'Other Form Single Type'
            gen = select_generation
#             print(kdex,ndex,poke_name, type1,type2)
#             print(pokemon_info_values.text.strip().split('\n'))
#             print('####')

        ## Pokemons that are other regional forms and has 2 types
        elif len(pokemon_raw_info) == 8:
            ldex = pokemon_raw_info[0]
            ndex = pokemon_raw_info[1]
            poke_name = pokemon_raw_info[4]
            type1 = pokemon_raw_info[6]
            type2 = pokemon_raw_info[7]
            categ = 'Other Form Multi Type'
            gen = select_generation
#             print(kdex,ndex,poke_name, type1,type2)
#             print(pokemon_info_values.text.strip().split('\n'))
#             print('####')

        ## Orig pokemon list with single typing
        elif len(pokemon_raw_info) == 9:
            ldex = pokemon_raw_info[0]
            ndex = pokemon_raw_info[2]
            poke_name = pokemon_raw_info[6]
            type1 = pokemon_raw_info[8]
            type2 = ''
            categ = 'Orig Form Single Type'
            gen = select_generation
#             print(kdex,ndex,poke_name, type1,type2)
#             print(pokemon_info_values.text.strip().split('\n'))
#             print('####')


        # Orig pokemon list with multiple typing
        elif len(pokemon_raw_info) == 10:
            ldex = pokemon_raw_info[0]
            ndex = pokemon_raw_info[2]
            poke_name = pokemon_raw_info[6]
            type1 = pokemon_raw_info[8]
            type2 = pokemon_raw_info[9]
            categ = 'Orig Form Multi Type'
            gen = select_generation
#             print(kdex,ndex,poke_name, type1,type2)
#             print(pokemon_info_values.text.strip().split('\n'))
#             print('####')

        else:
            print('Check out elements containing ' + str(len(pokemon_raw_info)) + ' elements')
        
        # Saving as a tuple
        extracted_poke_info.append((ldex, ndex, poke_name, type1, type2, categ, gen))
        
    
    else:
        pass

    
    #print(pokemon_info_values)

In [6]:
def generate_json(select_generation):
    generate_pd(select_generation)
    df_pokemon_list = pd.DataFrame(extracted_poke_info)
    df_pokemon_list.columns = ['Ldex', 'Ndex', 'Pokemon', 'Type 1', 'Type 2', 'Category', 'Generation']
    print(df_pokemon_list)
    df_pokemon_list.to_json('Dy_Velarde_pokemon.json',orient='index')

In [7]:
#testing
extracted_poke_info = []
generate_json(8)
pd.read_json('Dy_Velarde_pokemon.json', orient='index')

     Ldex  Ndex      Pokemon    Type 1  Type 2               Category  \
0    #001  #810      Grookey     Grass          Orig Form Single Type   
1    #002  #811     Thwackey     Grass          Orig Form Single Type   
2    #003  #812    Rillaboom     Grass          Orig Form Single Type   
3    #004  #813    Scorbunny      Fire          Orig Form Single Type   
4    #005  #814       Raboot      Fire          Orig Form Single Type   
..    ...   ...          ...       ...     ...                    ...   
96   #---  #901     Ursaluna    Ground  Normal   Orig Form Multi Type   
97   #---  #902  Basculegion     Water   Ghost   Orig Form Multi Type   
98   #---  #903     Sneasler  Fighting  Poison   Orig Form Multi Type   
99   #---  #904     Overqwil      Dark  Poison   Orig Form Multi Type   
100  #---  #905     Enamorus     Fairy  Flying   Orig Form Multi Type   

     Generation  
0             8  
1             8  
2             8  
3             8  
4             8  
..          ...

ValueError: Expected object or value